In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [2]:
dataframe = pd.read_csv("csv/82000278_Toamnei_CO2_2021_7_SHORT.csv")
dataframe.head()

,time,latitude,longitude,altitude,co2
0,1625086823,45.651464,25.615426,100,553
1,1625086883,45.651464,25.615426,100,551
2,1625086943,45.651464,25.615426,100,551
3,1625087003,45.651464,25.615426,100,558
4,1625087063,45.651464,25.615426,100,559


In [4]:
X = dataframe.select_dtypes(include="number").drop("co2", axis=1)
y = dataframe.co2

X_test = dataframe.select_dtypes(include="number")

X_test.head()

,time,latitude,longitude,altitude,co2
0,1625086823,45.651464,25.615426,100,553
1,1625086883,45.651464,25.615426,100,551
2,1625086943,45.651464,25.615426,100,551
3,1625087003,45.651464,25.615426,100,558
4,1625087063,45.651464,25.615426,100,559


In [5]:
from sklearn.impute import SimpleImputer

# Impute both train and test sets
imputer = SimpleImputer(strategy="mean")
X = imputer.fit_transform(X)
X_test = imputer.fit_transform(X_test)

In [6]:
%%time

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3)

# Fit a base model
forest = RandomForestRegressor()

_ = forest.fit(X_train, y_train)

print(f"R2 for training set: {forest.score(X_train, y_train)}")
print(f"R2 for validation set: {forest.score(X_valid, y_valid)}\n")


R2 for training set: 0.9991094267723697
R2 for validation set: 0.9977540250367847

CPU times: total: 1.08 s
Wall time: 1.51 s


In [7]:
forest.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'squared_error',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [8]:
n_estimators = np.arange(100, 2000, step=100)
max_features = ["auto", "sqrt", "log2"]
max_depth = list(np.arange(10, 100, step=10)) + [None]
min_samples_split = np.arange(2, 10, step=2)
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]

param_grid = {
    "n_estimators": n_estimators,
    "max_features": max_features,
    "max_depth": max_depth,
    "min_samples_split": min_samples_split,
    "min_samples_leaf": min_samples_leaf,
    "bootstrap": bootstrap,
}

param_grid

{'n_estimators': array([ 100,  200,  300,  400,  500,  600,  700,  800,  900, 1000, 1100,
        1200, 1300, 1400, 1500, 1600, 1700, 1800, 1900]),
 'max_features': ['auto', 'sqrt', 'log2'],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, None],
 'min_samples_split': array([2, 4, 6, 8]),
 'min_samples_leaf': [1, 2, 4],
 'bootstrap': [True, False]}

In [9]:
from sklearn.model_selection import RandomizedSearchCV

forest = RandomForestRegressor()

random_cv = RandomizedSearchCV(
    forest, param_grid, n_iter=100, cv=3, scoring="r2", n_jobs=-1
)

In [10]:
%%time

_ = random_cv.fit(X, y)

print("Best params:\n")
print(random_cv.best_params_)

random_cv.best_score_

Best params:

{'n_estimators': 700, 'min_samples_split': 6, 'min_samples_leaf': 4, 'max_features': 'sqrt', 'max_depth': 60, 'bootstrap': True}
CPU times: total: 7.47 s
Wall time: 11min 37s


-1.1387588318038444

In [11]:
n_iterations = 1

for value in param_grid.values():
    n_iterations *= len(value)
    
n_iterations

13680

In [32]:
new_params = {
    "n_estimators": [600, 700, 800],
    "max_features": ['sqrt'],
    "max_depth": [55, 60, 65],
    "min_samples_split": [4, 6, 8],
    "min_samples_leaf": [3, 4, 5],
    "bootstrap": [True],
}

new_params

{'n_estimators': [600, 700, 800],
 'max_features': ['sqrt'],
 'max_depth': [55, 60, 65],
 'min_samples_split': [4, 6, 8],
 'min_samples_leaf': [3, 4, 5],
 'bootstrap': [True]}

In [16]:
n_iterations = 1

for value in new_params.values():
    n_iterations *= len(value)
    
n_iterations

81

In [17]:
from sklearn.model_selection import GridSearchCV

forest = RandomForestRegressor()

grid_cv = GridSearchCV(forest, new_params, n_jobs=-1)

In [18]:
%%time

_ = grid_cv.fit(X, y)

print('Best params:\n')
print(grid_cv.best_params_, '\n')

Best params:

{'bootstrap': True, 'max_depth': 55, 'max_features': 'sqrt', 'min_samples_leaf': 5, 'min_samples_split': 6, 'n_estimators': 600} 

CPU times: total: 6.36 s
Wall time: 8min 38s


In [19]:
grid_cv.best_score_

-1.2896261762348877